<a href="https://colab.research.google.com/github/Smart-Pigs/Cow_Food_Consumption/blob/main/Data_Cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Mount Google Drive 
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

Mounted at /content/gdrive


In [2]:
#Change path
import os
path = '/content/gdrive/MyDrive/Cow_eating_label/'
os.chdir(path)

#concatenate for outliers detection 

In [ ]:
import glob
import numpy as np
import pandas as pd

#remove frist two columns of counts
df = pd.read_csv('Cow6065/Final/cow6065_1pm_to_5pm_1.csv')
column_names = df.columns.tolist()[2:]
print(column_names)

#prepare dataset
folder = 'Cow6065/Final'
output_df='Cow6065/Final/output_df'
df = pd.DataFrame()
for file in os.listdir(folder):
  if file.endswith(".csv"):
    file_path = os.path.join(folder, file)
    file_df = pd.read_csv(file_path)
    file_df=file_df[column_names]
    file_df["label"] = file_df["label"].replace("grazing", "eating")
    file_df["label"] = file_df["label"].fillna("not_eating")
    df = pd.concat([df, file_df], ignore_index=True)
    print(f'Add {file}')
output = os.path.join(output_df, '6065_all.csv')
df.to_csv(output, index=False)
    


['acc_x_leg', 'acc_y_leg', 'acc_z_leg', 'gyro_x_leg', 'gyro_y_leg', 'gyro_z_leg', 'acc_x_maxilar', 'acc_y_maxilar', 'acc_z_maxilar', 'gyro_x_maxilar', 'gyro_y_maxilar', 'gyro_z_maxilar', 'acc_x_nose', 'acc_y_nose', 'acc_z_nose', 'gyro_x_nose', 'gyro_y_nose', 'gyro_z_nose', 'label']
Add cow6065_1pm_to_5pm_1.csv


<ipython-input-11-e17758a81871>:17: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  file_df = pd.read_csv(file_path)
<ipython-input-11-e17758a81871>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  file_df["label"] = file_df["label"].replace("grazing", "eating")
<ipython-input-11-e17758a81871>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  file_df["label"] = file_df["label"].fillna("not_eating")


Add cow6065_5pm_to_9pm.csv
Add cow6065_9am_to_1pm_1.csv


<ipython-input-11-e17758a81871>:17: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  file_df = pd.read_csv(file_path)


Add cow6065_9am_to_1pm_2.csv
Add cow6065_1pm_to_5pm_2.csv
Add cow6065_9pm_to_12am.csv


In [ ]:
# 1.5IQR
def Detect_outlier_IQR(df):
  Q1 = df.quantile(0.25)
  Q3 = df.quantile(0.75)
  IQR = Q3 - Q1
  down_threshold = Q1-1.5*IQR
  up_threshold = Q3+1.5*IQR
  return up_threshold,down_threshold


#label set
labels=set(df['label'])
df=pd.read_csv('Cow6065/Final/output_df/6065_all.csv')
ouliers=0
out_percent=0
for label in labels:
  data_item = df[df['label'] == label]
  print(f"Label: {label}")
  for col in data_item.columns[:-1]:
    col_threshold = Detect_outlier_IQR(data_item[col])
    outliers = data_item[(data_item[col]<col_threshold[1])|(data_item[col]>col_threshold[0])].shape[0]
    print(f"{col}: {col_threshold}")
  print("\n")
out_percent=outliers/df.shape[0] 
print(out_percent) 



Label: eating
acc_x_leg: (2.14434814453125, -2.52117919921875)
acc_y_leg: (-9.695922851562502, -10.021313476562499)
acc_z_leg: (0.08732910156249984, -1.8171630859375)
gyro_x_leg: (0.8079268485307691, -1.0213414877653122)
gyro_y_leg: (0.9146341681480408, -1.5243902802467346)
gyro_z_leg: (2.14939022064209, -1.3871951103210447)
acc_x_maxilar: (12.22906494140625, 3.6109985351562517)
acc_y_maxilar: (9.8861328125, -2.6318359374999996)
acc_z_maxilar: (0.11962890624999956, -8.67548828125)
gyro_x_maxilar: (70.67073059082031, -70.06097412109375)
gyro_y_maxilar: (41.15853834152222, -39.81707429885864)
gyro_z_maxilar: (49.771342277526855, -52.05792713165283)
acc_x_nose: (9.349597167968751, -2.5797973632812505)
acc_y_nose: (-2.9895263671875023, -13.765698242187497)
acc_z_nose: (10.08890380859375, -5.07525634765625)
gyro_x_nose: (59.71036624908447, -58.70426845550537)
gyro_y_nose: (44.1463418006897, -45.365853786468506)
gyro_z_nose: (35.21341514587402, -36.0060977935791)


Label: ruminating
acc_x_le

#Back to original files and remove outliers

In [26]:
#Label: eating
thres_eat={
    'acc_x_leg': (2.14434814453125, -2.52117919921875),
    'acc_y_leg': (-9.695922851562502, -10.021313476562499),
    'acc_z_leg': (0.08732910156249984, -1.8171630859375),
    'gyro_x_leg': (0.8079268485307691, -1.0213414877653122),
    'gyro_y_leg': (0.9146341681480408, -1.5243902802467346),
    'gyro_z_leg': (2.14939022064209, -1.3871951103210447),
    'acc_x_maxilar': (12.22906494140625, 3.6109985351562517),
    'acc_y_maxilar': (9.8861328125, -2.6318359374999996),
    'acc_z_maxilar': (0.11962890624999956, -8.67548828125),
    'gyro_x_maxilar': (70.67073059082031, -70.06097412109375),
    'gyro_y_maxilar': (41.15853834152222, -39.81707429885864),
    'gyro_z_maxilar': (49.771342277526855, -52.05792713165283),
    'acc_x_nose': (9.349597167968751, -2.5797973632812505),
    'acc_y_nose': (-2.9895263671875023, -13.765698242187497),
    'acc_z_nose': (10.08890380859375, -5.07525634765625),
    'gyro_x_nose': (59.71036624908447, -58.70426845550537),
    'gyro_y_nose': (44.1463418006897, -45.365853786468506),
    'gyro_z_nose': (35.21341514587402, -36.0060977935791)
}

#Label: ruminating
thres_rum = {
    'acc_x_leg': (0.14833984375000003, -0.59814453125),
    'acc_y_leg': (-9.831701660156249, -9.93218994140625),
    'acc_z_leg': (-0.13637695312499998, -1.6006347656250002),
    'gyro_x_leg': (0.8536585569381712, -1.097561001777649),
    'gyro_y_leg': (0.9298780709505085, -1.6310975998640063),
    'gyro_z_leg': (2.3628049343824387, -1.6615854054689407),
    'acc_x_maxilar': (12.290374755859377, -0.7707092285156261),
    'acc_y_maxilar': (9.742877197265624, 1.5195861816406269),
    'acc_z_maxilar': (-1.1891113281249992, -9.654052734375002),
    'gyro_x_maxilar': (75.8993935585022, -73.73475790023804),
    'gyro_y_maxilar': (47.22560977935791, -44.96951198577881),
    'gyro_z_maxilar': (55.80792856216431, -59.07012319564819),
    'acc_x_nose': (9.248211669921876, -1.310235595703126),
    'acc_y_nose': (-2.109057617187501, -13.751342773437498),
    'acc_z_nose': (8.512493896484376, -1.376031494140626),
    'gyro_x_nose': (60.533538579940796, -59.100610971450806),
    'gyro_y_nose': (48.6661593914032, -51.02896428108215),
    'gyro_z_nose': (36.364330530166626, -37.84298872947693)
}



#Label: not_eating
thres_neat = {
    'acc_x_leg': (2.3590820312500003, -3.5075195312500003),
    'acc_y_leg': (-9.5643310546875, -10.0906982421875),
    'acc_z_leg': (0.8595336914062502, -2.1311889648437505),
    'gyro_x_leg': (0.9603658765554428, -1.1128049045801163),
    'gyro_y_leg': (1.0365853458642957, -1.6463414281606672),
    'gyro_z_leg': (2.2256098091602325, -1.4329268634319303),
    'acc_x_maxilar': (12.182409667968752, -2.3788208007812504),
    'acc_y_maxilar': (11.632714843750001, 1.8805664062500003),
    'acc_z_maxilar': (-1.9320068359375, -7.7698974609375),
    'gyro_x_maxilar': (47.91158580780029, -47.33231830596924),
    'gyro_y_maxilar': (27.13414692878723, -26.036585569381714),
    'gyro_z_maxilar': (25.472561836242676, -25.868903160095215),
    'acc_x_nose': (7.879956054687502, 0.12800292968749938),
    'acc_y_nose': (-3.5505859375000015, -12.403124999999998),
    'acc_z_nose': (9.11392822265625, -2.2508178710937496),
    'gyro_x_nose': (31.417683362960815, -32.85061001777649),
    'gyro_y_nose': (32.46951222419739, -33.87195086479187),
    'gyro_z_nose': (26.097561597824097, -26.58536648750305)
}



In [30]:
import glob
import numpy as np
import pandas as pd

Input_path = '/content/gdrive/MyDrive/Cow_eating_label/Cow6065/Final/'
Output_path = '/content/gdrive/MyDrive/Cow_eating_label/Cow6065/Final/clean/'

df = pd.DataFrame()
for file_name in os.listdir(Input_path):
  if file_name.endswith('.csv'):
    file_path = os.path.join(Input_path, file_name)
    df = pd.read_csv(file_path)
    df = df[df.columns.tolist()[2:]]
    for label, threshold in [('eating', thres_eat), ('ruminating', thres_rum), ('not_eating', thres_neat)]:
      mask = df['label'] == label
      for col in df.columns[:-1]: 
        if col in threshold:
          # Remove outliers
          lower_thresh, upper_thresh = threshold[col]
          outside_thresh_mask = (df[col] < lower_thresh) | (df[col] > upper_thresh)
          df.loc[outside_thresh_mask & mask, col] = np.nan
          # Interpolate  
          df.loc[mask, col] = df.loc[mask, col].interpolate(method='linear', limit_direction='both')
          # df_new = df_new.append(df)
    clean_file_path = os.path.join(Output_path, f'{file_name}')
    df.to_csv(clean_file_path, index=False)
    print(f'Saved {clean_file_path}')
    df = pd.DataFrame()









Saved /content/gdrive/MyDrive/Cow_eating_label/Cow6065/Final/clean/cow6065_1pm_to_5pm_1.csv


<ipython-input-30-5507bad65eab>:12: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


Saved /content/gdrive/MyDrive/Cow_eating_label/Cow6065/Final/clean/cow6065_5pm_to_9pm.csv
Saved /content/gdrive/MyDrive/Cow_eating_label/Cow6065/Final/clean/cow6065_9am_to_1pm_1.csv


<ipython-input-30-5507bad65eab>:12: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


Saved /content/gdrive/MyDrive/Cow_eating_label/Cow6065/Final/clean/cow6065_9am_to_1pm_2.csv
Saved /content/gdrive/MyDrive/Cow_eating_label/Cow6065/Final/clean/cow6065_1pm_to_5pm_2.csv
Saved /content/gdrive/MyDrive/Cow_eating_label/Cow6065/Final/clean/cow6065_9pm_to_12am.csv
